### Instalando pacotes

In [68]:
!pip install openpyxl
!pip install pandas
!pip install numpy


[notice] A new release of pip is available: 24.3.1 -> 25.3
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.3.1 -> 25.3
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.3.1 -> 25.3
[notice] To update, run: pip install --upgrade pip


In [69]:
import pandas as pd
import numpy as np
import openpyxl

# Avaliação 03
Grupo 5:
*   Allane
*   Item da lista



### Instanciando os grafos
Estamos usando DataFrames da biblioteca pandas para representar as matrizes de adjacencia dos grafos.

In [70]:
matriz_problema_km = pd.read_excel("./PCV__Matriz_do_problema.xlsx", sheet_name="Km")
matriz_problema_min = pd.read_excel("./PCV__Matriz_do_problema.xlsx", sheet_name="Min")
cidades = pd.read_excel("./PCV__Matriz_do_problema.xlsx", sheet_name="Cidades")


In [71]:
# Estamos considerando o label numerico
def vizinho_mais_proximo(grafo : pd.DataFrame, vertice_inicial : int | str):
    # Criando conjunto de vertices nao visitados
    # o conjunto contem os indices referentes ao vertice na matriz problema
    vertices_nao_visitados = set(grafo.index)
    percurso = [vertice_inicial]
    vertices_nao_visitados.remove(vertice_inicial)
    dist_total = 0
    v_atual = vertice_inicial
    while len(vertices_nao_visitados) > 0:
        # Obtem o label (numero) do vertice com menor custo
        idx_vizinho_mais_proximo = grafo[v_atual].loc[list(vertices_nao_visitados)].idxmin(skipna=True)
        prox = idx_vizinho_mais_proximo + 1
        dist_total += grafo[v_atual][idx_vizinho_mais_proximo]
        percurso.append(int(prox))
        v_atual = prox
        vertices_nao_visitados.remove(idx_vizinho_mais_proximo)
    dist_total += grafo[v_atual][vertice_inicial]
    percurso.append(vertice_inicial)
    return percurso, dist_total


## Busca por vizinho mais proximo

### Problema 1
Percurso por 48 cidades, partindo de ANGICOS, com funcao custo definida pela distancia.


In [72]:
print("Testando a busca por vizinho mais proximo")
caminho, custo = vizinho_mais_proximo(matriz_problema_km, 1)
print(f"Caminho: {caminho}")
print(f"Custo: {custo}")


Testando a busca por vizinho mais proximo
Caminho: [1, 8, 9, 10, 1, 3, 4, 23, 21, 22, 48, 40, 25, 17, 7, 19, 16, 34, 33, 20, 38, 37, 47, 39, 35, 42, 26, 32, 31, 36, 12, 11, 5, 14, 46, 29, 24, 43, 13, 41, 6, 27, 30, 44, 28, 18, 15, 45, 1]
Custo: 2492.0


### Estrutura da Busca Local

In [74]:
# Funca para obter o custo total a partir de uma solucao (caminho)
def calcular_custo(solucao, grafo):
    if len(solucao) <= 1:
        return
    custo = 0
    for prox in range(1, len(solucao)):
        v1 = solucao[prox - 1]
        v2 = solucao[prox]
        custo += grafo[prox][prox-1]
    return custo

# O argumento 'heuristica' eh o algoritmo da busca local que sera realizada (shift, swap etc).
# ele deve retornar uma vizinhanca - a lista de solucoes geradas
def busca_local_primeira_melhoria(solucao, custo_original, heuristica, grafo):
    custo_atual = custo_original
    solucao_atual = solucao
    melhoria = True
    while melhoria:
        melhoria = False
        for nova_solucao, custo in heuristica(solucao):
            if custo < custo_atual:
                solucao_atual = nova_solucao
                melhoria = True
                continue
    return solucao_atual, custo_atual

def busca_local_melhor_melhoria(solucao, custo_original, heuristica, grafo):
    custo_atual = melhor_custo = custo_original
    solucao_atual = melhor_solucao = solucao
    melhoria = True
    while melhoria:
        melhoria = False
        for nova_solucao in heuristica(solucao):
            novo_custo = calcular_custo(nova_solucao, grafo)
            if novo_custo < custo_atual:
                custo_atual = novo_custo
                solucao_atual = nova_solucao
        if custo_atual < melhor_custo:
            melhor_custo = custo_atual
            melhoria = True
    return melhor_solucao, melhor_custo
